# Model Context Protocol (MCP)

[Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction)은 애플리케이션이 LLM에 도구와 컨텍스트를 제공하는 방법을 표준화하는 오픈 프로토콜입니다.

LangChain 에이전트는 `langchain-mcp-adapters` 라이브러리를 사용하여 MCP 서버에 정의된 도구를 사용할 수 있습니다.

## MCP의 주요 특징

- **표준화된 인터페이스**: 다양한 도구와 서비스를 일관된 방식으로 연결
- **다중 전송 방식**: stdio, HTTP, SSE 등 다양한 통신 방식 지원
- **상태 관리**: Stateless 및 Stateful 도구 사용 지원
- **확장성**: 커스텀 MCP 서버 생성 가능

## 설치

MCP 도구를 사용하기 위해 필요한 라이브러리를 설치합니다.

```bash
pip install langchain-mcp-adapters
pip install mcp
```

## 사전 준비

환경 변수를 설정합니다.

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

## 전송 타입 (Transport Types)

MCP는 클라이언트-서버 통신을 위한 다양한 전송 메커니즘을 지원합니다:

### 1. stdio

- 클라이언트가 서버를 서브프로세스로 실행하고 표준 입출력을 통해 통신
- 로컬 도구 및 간단한 설정에 적합

### 2. Streamable HTTP

- 서버가 독립적인 프로세스로 실행되어 HTTP 요청 처리
- 원격 연결 및 다중 클라이언트 지원

### 3. Server-Sent Events (SSE)

- 실시간 스트리밍 통신에 최적화된 Streamable HTTP의 변형

## 커스텀 MCP 서버 생성

먼저 테스트를 위한 커스텀 MCP 서버를 생성합니다.

### Math Server (stdio)

아래 코드를 `math_server.py` 파일로 저장합니다.

In [ ]:
# math_server.py 파일 내용
math_server_code = '''
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Math")

@mcp.tool()
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

@mcp.tool()
def subtract(a: int, b: int) -> int:
    """Subtract b from a"""
    return a - b

if __name__ == "__main__":
    mcp.run(transport="stdio")
'''

# 파일로 저장
with open('math_server.py', 'w') as f:
    f.write(math_server_code)

print("math_server.py created successfully")

### Weather Server (Streamable HTTP)

아래 코드를 `weather_server.py` 파일로 저장하고 별도로 실행해야 합니다.

```bash
python weather_server.py
```

In [ ]:
# weather_server.py 파일 내용
weather_server_code = '''
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Weather")

@mcp.tool()
async def get_weather(location: str) -> str:
    """Get weather for location."""
    # 실제로는 API 호출을 수행
    weather_data = {
        "new york": "It's sunny in New York, 22°C",
        "london": "It's rainy in London, 15°C",
        "tokyo": "It's cloudy in Tokyo, 18°C",
        "seoul": "It's clear in Seoul, 20°C"
    }
    
    location_lower = location.lower()
    for city, weather in weather_data.items():
        if city in location_lower:
            return weather
    
    return f"Weather data not available for {location}"

@mcp.tool()
async def get_forecast(location: str, days: int = 3) -> str:
    """Get weather forecast for location."""
    return f"{days}-day forecast for {location}: Sunny, Cloudy, Rainy"

if __name__ == "__main__":
    mcp.run(transport="streamable-http", port=8000)
'''

# 파일로 저장
with open('weather_server.py', 'w') as f:
    f.write(weather_server_code)

print("weather_server.py created successfully")
print("Run 'python weather_server.py' in a separate terminal to start the server")

## MCP 도구 사용

`langchain-mcp-adapters`를 사용하면 에이전트가 하나 이상의 MCP 서버에 정의된 도구를 사용할 수 있습니다.

### 단일 MCP 서버 사용 (stdio)

In [ ]:
import os
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# Math 서버만 사용 (stdio 전송)
client = MultiServerMCPClient(
    {
        "math": {
            "transport": "stdio",  # 로컬 서브프로세스 통신
            "command": "python",
            "args": [os.path.abspath("math_server.py")],  # 절대 경로
        }
    }
)

# MCP 서버에서 도구 가져오기
tools = await client.get_tools()

print(f"Available tools: {[tool.name for tool in tools]}")

# 에이전트 생성
model = ChatOpenAI(model="gpt-4.1-mini")
agent = create_agent(model, tools)

# 수학 연산 수행
math_response = await agent.ainvoke(
    {"messages": [{"role": "user", "content": "what's (3 + 5) x 12?"}]}
)

print("\nMath result:")
print(math_response["messages"][-1].content)

### 다중 MCP 서버 사용

여러 MCP 서버를 동시에 사용할 수 있습니다.

**주의**: Weather 서버를 사용하려면 먼저 별도 터미널에서 `python weather_server.py`를 실행해야 합니다.

In [ ]:
# 다중 MCP 서버 설정
client = MultiServerMCPClient(
    {
        "math": {
            "transport": "stdio",
            "command": "python",
            "args": [os.path.abspath("math_server.py")],
        },
        "weather": {
            "transport": "streamable_http",  # HTTP 기반 원격 서버
            "url": "http://localhost:8000/mcp",  # Weather 서버가 포트 8000에서 실행 중이어야 함
        }
    }
)

# 모든 서버에서 도구 가져오기
tools = await client.get_tools()

print(f"Available tools from multiple servers: {[tool.name for tool in tools]}")

agent = create_agent(model, tools)

# Math 서버 도구 사용
math_response = await agent.ainvoke(
    {"messages": [{"role": "user", "content": "Calculate 15 + 27"}]}
)
print("\nMath response:")
print(math_response["messages"][-1].content)

# Weather 서버 도구 사용 (서버가 실행 중인 경우)
try:
    weather_response = await agent.ainvoke(
        {"messages": [{"role": "user", "content": "What is the weather in Seoul?"}]}
    )
    print("\nWeather response:")
    print(weather_response["messages"][-1].content)
except Exception as e:
    print(f"\nWeather server error: {e}")
    print("Make sure to run 'python weather_server.py' in a separate terminal")

### 복합 작업 수행

여러 서버의 도구를 조합하여 복잡한 작업을 수행할 수 있습니다.

In [ ]:
# 복합 질문
complex_response = await agent.ainvoke({
    "messages": [{
        "role": "user",
        "content": "First, calculate 10 x 5. Then tell me the weather in Tokyo."
    }]
})

print("Complex task result:")
print(complex_response["messages"][-1].content)

## Stateful 도구 사용

도구 호출 간에 컨텍스트를 유지하는 Stateful 서버의 경우 `client.session()`을 사용하여 영구적인 `ClientSession`을 생성합니다.

In [ ]:
from langchain_mcp_adapters.tools import load_mcp_tools

client = MultiServerMCPClient(
    {
        "math": {
            "transport": "stdio",
            "command": "python",
            "args": [os.path.abspath("math_server.py")],
        }
    }
)

# 영구 세션 사용
async with client.session("math") as session:
    tools = await load_mcp_tools(session)
    
    agent = create_agent(model, tools)
    
    # 세션 내에서 여러 작업 수행
    result1 = await agent.ainvoke(
        {"messages": [{"role": "user", "content": "Add 5 and 3"}]}
    )
    print("Result 1:", result1["messages"][-1].content)
    
    result2 = await agent.ainvoke(
        {"messages": [{"role": "user", "content": "Multiply 4 and 7"}]}
    )
    print("Result 2:", result2["messages"][-1].content)

## 실용적인 예제: 계산기와 날씨 봇

MCP를 활용한 실용적인 봇 예제입니다.

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# MCP 클라이언트 설정
client = MultiServerMCPClient(
    {
        "math": {
            "transport": "stdio",
            "command": "python",
            "args": [os.path.abspath("math_server.py")],
        }
    }
)

tools = await client.get_tools()
agent = create_agent(
    ChatOpenAI(model="gpt-4.1-mini"),
    tools,
    system_prompt="You are a helpful assistant that can perform calculations and provide weather information."
)

# 사용자 질문 처리
user_queries = [
    "Calculate the total cost: I bought 3 items at $15 each and 2 items at $25 each",
    "What's 123 multiplied by 456?",
    "If I have 100 dollars and spend 37, how much do I have left?",
]

print("=== Calculator Bot ===")
for query in user_queries:
    print(f"\nUser: {query}")
    response = await agent.ainvoke(
        {"messages": [{"role": "user", "content": query}]}
    )
    print(f"Bot: {response['messages'][-1].content}")

## 고급 MCP 서버: 상태 관리

상태를 유지하는 고급 MCP 서버 예제입니다.

In [ ]:
# calculator_server.py - 계산 히스토리를 유지하는 서버
calculator_server_code = '''
from mcp.server.fastmcp import FastMCP
from typing import List

mcp = FastMCP("Calculator")

# 계산 히스토리를 저장할 전역 변수
calculation_history: List[str] = []

@mcp.tool()
def calculate(expression: str) -> str:
    """Calculate mathematical expression and save to history."""
    try:
        result = eval(expression)  # 주의: 실제 프로덕션에서는 안전한 평가 방법 사용
        history_entry = f"{expression} = {result}"
        calculation_history.append(history_entry)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

@mcp.tool()
def get_history() -> str:
    """Get calculation history."""
    if not calculation_history:
        return "No calculations yet"
    return "\\n".join(calculation_history)

@mcp.tool()
def clear_history() -> str:
    """Clear calculation history."""
    calculation_history.clear()
    return "History cleared"

if __name__ == "__main__":
    mcp.run(transport="stdio")
'''

with open('calculator_server.py', 'w') as f:
    f.write(calculator_server_code)

print("calculator_server.py created successfully")

### 상태 유지 서버 사용

In [ ]:
# Calculator 서버 사용
client = MultiServerMCPClient(
    {
        "calculator": {
            "transport": "stdio",
            "command": "python",
            "args": [os.path.abspath("calculator_server.py")],
        }
    }
)

# 영구 세션으로 히스토리 유지
async with client.session("calculator") as session:
    tools = await load_mcp_tools(session)
    agent = create_agent(ChatOpenAI(model="gpt-4.1-mini"), tools)
    
    # 여러 계산 수행
    print("=== Calculation 1 ===")
    result1 = await agent.ainvoke(
        {"messages": [{"role": "user", "content": "Calculate 10 + 20"}]}
    )
    print(result1["messages"][-1].content)
    
    print("\n=== Calculation 2 ===")
    result2 = await agent.ainvoke(
        {"messages": [{"role": "user", "content": "Calculate 5 * 8"}]}
    )
    print(result2["messages"][-1].content)
    
    print("\n=== Get History ===")
    history = await agent.ainvoke(
        {"messages": [{"role": "user", "content": "Show me the calculation history"}]}
    )
    print(history["messages"][-1].content)

## MCP vs 일반 도구

### MCP의 장점

1. **표준화**: 일관된 인터페이스로 다양한 서비스 연결
2. **재사용성**: 한 번 정의한 MCP 서버를 여러 에이전트에서 사용
3. **독립성**: 서버가 독립적으로 실행되어 유지보수 용이
4. **확장성**: 새로운 도구 추가가 간편

### 일반 도구가 더 나은 경우

1. **간단한 작업**: 복잡한 서버 설정이 필요 없는 단순 함수
2. **빠른 프로토타이핑**: MCP 서버 설정 없이 빠르게 테스트
3. **긴밀한 통합**: 에이전트 코드와 강하게 결합된 로직

## 주의사항

### 1. Stateless vs Stateful

`MultiServerMCPClient`는 기본적으로 **stateless**입니다. 각 도구 호출은 새로운 MCP `ClientSession`을 생성하고, 도구를 실행한 후 정리합니다.

상태를 유지해야 하는 경우 `client.session()`을 사용하세요.

In [ ]:
# Stateless (기본)
tools = await client.get_tools()
agent = create_agent(model, tools)
# 각 호출마다 새로운 세션

# Stateful (세션 유지)
async with client.session("server_name") as session:
    tools = await load_mcp_tools(session)
    agent = create_agent(model, tools)
    # 세션 내에서 상태 유지

### 2. 에러 처리

MCP 서버가 실행 중이지 않거나 연결할 수 없는 경우 적절한 에러 처리가 필요합니다.

In [ ]:
try:
    client = MultiServerMCPClient({
        "weather": {
            "transport": "streamable_http",
            "url": "http://localhost:8000/mcp",
        }
    })
    tools = await client.get_tools()
except Exception as e:
    print(f"Failed to connect to MCP server: {e}")
    print("Make sure the server is running")

### 3. 보안

- stdio 전송은 로컬 서브프로세스만 신뢰해야 합니다
- HTTP 전송은 적절한 인증 및 암호화를 사용해야 합니다
- `eval()` 같은 위험한 함수는 피하세요

## 정리

MCP를 사용하면:

1. **표준화된 방식**으로 다양한 도구와 서비스를 LLM에 연결할 수 있습니다
2. **stdio, HTTP, SSE** 등 다양한 전송 방식을 선택할 수 있습니다
3. **Stateless 및 Stateful** 도구 사용을 모두 지원합니다
4. **커스텀 MCP 서버**를 쉽게 생성하고 배포할 수 있습니다

MCP는 LLM 애플리케이션에서 도구와 컨텍스트를 관리하는 강력하고 표준화된 방법을 제공합니다.